This is the first part of the final exam for AcF 351b: Python Stream.   

Students are expected to act according to the highest ethical standards. All students enrolled at Lancaster University are to perform their academic work according to standards set by faculty members, departments, schools and colleges of the university; and cheating and plagiarism constitute fraudulent misrepresentation for which appropriate sanctions are warranted and will be applied. Please note that any form of violation of the following rules will be treated as plagiarism

1. Answer the questions yourself without asking others for assistance. This is a test of your ability of data science and computer programming. 
2. Do not share the questions or your answers with anyone. This includes posting the questions or your solutions publicly on services like quora, stackoverflow, or github.

**We will run a system to detect any kind of plagiarism, e.g., coding scripts with high similarities.**



**Do NOT erase the `#export` at the top of any cells as it is used by `notebook2script.py` to extract cells for submission.**

# Import modules. 
## Do NOT change the following cell!

In [1]:
#export
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## If you need extra modules, use the following cell to import them.

In [2]:
#export
# imported extra moduels:
import csv
import time
from datetime import datetime
import itertools
import math

# Section 1: Basic Data Science

In this section, you will be asked to answer questions regarding WiFi hotspot locations in NYC.  
Please make sure that the dataset entitled _"NYC_Wi-Fi_Hotspot_Locations.csv"_ is in the same folder as the Jupyter Notebook.   
Each row in the data represents one reported WiFi hotspot.  
A data dictionary is also provided. 



The following script reads the csv file into the memory, and stores it into a dataframe called _df_. Do not change it

In [3]:
#export
# if connected to the internet, import the dataset from the internet address
try:
    df = pd.read_csv("https://frankxu1987.weebly.com/uploads/6/2/5/8/62583677/nyc_wi-fi_hotspot_locations.csv")
# otherwise, import the dataset from the local .csv file
except:
    df = pd.read_csv("NYC_Wi-Fi_Hotspot_Locations.csv")    

### Please answer the following 8 questions. For each question,
1. Please write down the script used to compute your response in the ***Code*** Cell; Conclude your scripts with a final ***print()*** function to print out final numeric answers. **Important: Make sure your scripts are executable !!!**
2. Please fill out the final numeric answers in the cells at the end of the section. See below.

#### Question 1.1: How many unique providers are there? (10 pts)


In [4]:
#export
# Code script for Q 1.1
# Write your code script below
p = df["Provider"]
print(len(set(p)))
print()

17



#### Question 1.2: What fraction of WiFi hotspots are in parks? For simplicity, you can consider a park a place where the name of the location where the WiFi is located contains the word "park".? (10 pts)


In [5]:
#export
# Code script for Q 1.2
# Write your code script below 
p = 0
for i in df["Location"]:
    if i.find("Park") != -1:  
        p += 1
rate = p / len(df["Location"]) * 100
print(rate, "%") 
print()

3.5854172943657723 %



#### Question 1.3: How are WiFi hotspots distributed across neighborhoods? For this question, calculate the number of WiFi hotspots per capita for each Neighborhood Tabulation Area (NTA). Exclude NTAs with less than 30 reported WiFi hotspots. Report the [interquartile range](https://en.wikipedia.org/wiki/Interquartile_range) of the averages. 

**For population data for each NTA, use this [dataset](https://data.cityofnewyork.us/api/views/rnsn-acs2/rows.csv); information on the dataset is found [here](https://data.cityofnewyork.us/City-Government/Census-Demographics-at-the-Neighborhood-Tabulation/rnsn-acs2). Use the population data for the column corresponding to 2010. (10 pts)** 


In [6]:
#export
# Code script for Q 1.3
# Hint: you probably find the following line useful ***
df_nta= pd.read_csv("https://data.cityofnewyork.us/api/views/rnsn-acs2/rows.csv")
# Write your code script below 

mapping = {}

for nta in df_nta["Geographic Area - Neighborhood Tabulation Area (NTA)* Name"]:
    mapping[nta] = 0

NTA = df["Neighborhood Tabulation Area (NTA)"]

for i in NTA:
    if i in mapping:
        mapping[i] += 1
        
res = {k: v for k, v in mapping.items() if v > 30}

print(sum(res.values()) / len(res))

60.43333333333333


**Question 1.4: The dataset contains information on the date the hotspot was activated. What fraction of all activations occurred on the day of week that had the most activations? In other words, if Monday had the most activations, what fraction of activations occurred on Monday? Note: there are some dates that don't make sense. Ignore them for the analysis. (10 pts)**  


In [7]:
#export
# Code script for Q 1.4
# Write your code script below 

today_str = datetime.now().strftime("%m/%d/%Y")
today = time.strptime(today_str, "%m/%d/%Y")
datelist = [0] * 7
dates = df["Activated"]
for date_str in dates:
    date = time.strptime(date_str, "%m/%d/%Y")
    if date > today:
        continue

    week = datetime.strptime(date_str, "%m/%d/%Y").weekday()
    datelist[week] += 1

print(max(datelist) / sum(datelist))
print()

0.24368825466520308



#### Question 1.5: How many WiFi hotspots are there by the second most common provider in the Bronx? (10 pts) 


In [8]:
#export
# Code script for Q 1.5
# Write your code script below 

citys = df["City"]
provider = df["Provider"]
cp_dict = {}

for c, p in zip(citys, provider):
    if c == "Bronx":
        if p in cp_dict:
            cp_dict[p] += 1
        else:
            cp_dict[p] = 1

cp_dict = sorted(cp_dict.items(), key=lambda x: x[1], reverse=True)
if len(cp_dict) >= 2:
    print(cp_dict[1][1])
    print()

120



#### Question 1.6: What is the probability that a WiFi hotspot is free (without any limitations) given that it's not in a library? For this question, pull the location data based on the "Location_T" field.  (10 pts)
 

In [9]:
#export
# Code script for Q 1.6
# Write your code script below 
Type = df["Type"]
Location_T = df["Location_T"]
s, f = 0, 0
for L, T in zip(Location_T, Type):
    if L == "Library":
        continue
    s += 1
    if T == "Free":
        f += 1
        
print(f/s)
print()

0.8122383252818035



#### Question 1.7: How far must one travel from one hotspot to another? For this question, report the median distance, in feet, of the average distance between each hotspot to the nearest 3 hotspots. For your distance calculation, calculate the distance ["as the crow flies"](https://en.wikipedia.org/wiki/As_the_crow_flies). For simplicity, please use the [spherical Earth projected to a plane equation](https://en.wikipedia.org/wiki/Geographical_distance#Spherical_Earth_projected_to_a_plane) for calculating distances. Use the radius of the Earth as 6371 km.  (10 pts)
**Remember, report your answer in feet.**  


In [10]:
#export
# Code script for Q 1.7
# Write your code script below 

def distance(Latitude, Longitude):
    R = 6371
    fai = math.pa / 180
    

def distance(Latitude1, Longitude1, Latitude2, Longitude2):
    R = 6371
    fai = math.pi / 180
    derA = Latitude1 - Latitude2
    derO = Longitude1 - Longitude2
    return R * (((derA * fai) ** 2 + (derO * fai) ** 2) ** 0.5)

Latitudes = df["Latitude"]
Longitudes = df["Longitude"]

for la, lo in zip(Latitudes, Longitudes):
    dis = []
    for a, o in zip(Latitudes, Longitudes):
        dis.append(distance(la, lo, a, o))

    dis = sorted(dis)
    mid = dis[2]
    avg = (dis[1] + dis[2] + dis[3]) / 3
    
    print("point: ", la, lo, "mid: ", mid, "avg: ", avg)


point:  40.6748599999 -73.7841200005 mid:  0.22264827232657963 avg:  0.27092561565076184
point:  40.7475599996 -73.8181499997 mid:  0.4390228120420561 avg:  0.4511731237662134
point:  40.7119259997 -73.9406699994 mid:  0.4505850317667076 avg:  0.41829056277464477
point:  40.6899850001 -73.9919950004 mid:  0.00998777788478242 avg:  0.007296391345905089
point:  40.7646300002 -73.9661150001 mid:  0.18443256599437974 avg:  0.18589097907045157
point:  40.7424300003 -73.8115100003 mid:  0.3769639595228439 avg:  0.36399045653922046
point:  40.812944 -73.9416929996 mid:  0.09802329167220035 avg:  0.10225409486892678
point:  40.6873790941 -73.966030666 mid:  0.4374670669885906 avg:  0.33842725240556887
point:  40.6956020001 -73.9040840004 mid:  0.9275099440117945 avg:  0.8815142813882556
point:  40.6783340002 -73.9053160001 mid:  0.1245236117456386 avg:  0.15586071079339495
point:  40.8406639999 -73.8983720002 mid:  0.34466815139418294 avg:  0.3198318441010212
point:  40.6903899996 -74.01995999

point:  40.6808618482 -73.8464887284 mid:  1.0129786359626505 avg:  0.9710992033122062
point:  40.8241309998 -73.9424640006 mid:  0.0 avg:  0.0
point:  40.7470624208 -73.9442436779 mid:  0.07274306235966103 avg:  0.0922126395945689
point:  40.7412880001 -73.9046559997 mid:  0.3101357345533294 avg:  0.27990180636119555
point:  40.7656654522 -73.9574087323 mid:  0.06848993334305428 avg:  0.06646548922777971
point:  40.7410399998 -73.9978709994 mid:  0.04750686528862088 avg:  0.0903997632150939
point:  40.81841 -73.9267179996 mid:  0.16359508191900798 avg:  0.13294610907851698
point:  40.8037859998 -73.9461589995 mid:  0.19476665910136565 avg:  0.18502524808217866
point:  40.8342453801 -73.9411590502 mid:  0.0533922852093831 avg:  0.061575744565864045
point:  40.7661480002 -73.9797210006 mid:  0.17988099458956724 avg:  0.14332466623373377
point:  40.6874572699 -73.9819201796 mid:  0.06487892039589567 avg:  0.060451923381325136
point:  40.8620857697 -73.8995205706 mid:  0.1149360481740668 

point:  40.6419629997 -74.1020260005 mid:  0.0186502059241688 avg:  0.021129054191156408
point:  40.6861130002 -73.9739459997 mid:  0.10692656692346038 avg:  0.09744478188907851
point:  40.7102499999 -73.9979799999 mid:  0.0 avg:  0.0
point:  40.7472234296 -73.888492 mid:  0.06343209763190116 avg:  0.06798615592032821
point:  40.6782680804 -73.9497984494 mid:  0.08570705703041953 avg:  0.09678055315947581
point:  40.6955779998 -73.9839699996 mid:  0.09932303967604643 avg:  0.1390422528370847
point:  40.7476720001 -73.9769650002 mid:  0.06930617113376653 avg:  0.059122825256887196
point:  40.7402745363 -74.0019750591 mid:  0.06013827775263036 avg:  0.05387845166103398
point:  40.6599636231 -73.7399110132 mid:  3.3438938374368186 avg:  3.013294837746244
point:  40.7393355798 -73.9952642404 mid:  0.11724851081432469 avg:  0.10163771918626478
point:  40.7157269003 -74.0071850598 mid:  0.17214121037461264 avg:  0.14904909284446324
point:  40.6921780001 -73.9883039999 mid:  0.022423401888269

point:  40.5828240003 -73.8175779995 mid:  0.0347208711559237 avg:  0.036235720879801786
point:  40.687191 -73.7695589996 mid:  0.0 avg:  0.0
point:  40.6727480001 -73.9626939995 mid:  0.20518057707711343 avg:  0.21647495538247474
point:  40.6417853679 -74.0766457652 mid:  1.6357407152187136 avg:  1.2649219037817734
point:  40.74671976 -73.9859357197 mid:  0.03530530047470743 avg:  0.0483531233633128
point:  40.8397090001 -73.8874640004 mid:  0.16040223718535784 avg:  0.1178134496214226
point:  40.8059709999 -73.9405659999 mid:  0.07539096519441178 avg:  0.09568117420485
point:  40.811544 -73.9141379999 mid:  0.0 avg:  0.0
point:  40.6507380002 -73.9684290006 mid:  0.007312666529641522 avg:  0.01242464769051088
point:  40.7395470004 -73.7359200005 mid:  0.10291747747990969 avg:  0.11910166841763582
point:  40.7528042196 -74.0043127105 mid:  0.08347474389379599 avg:  0.08691889790012002
point:  40.6782855904 -73.9121847599 mid:  0.13107062712357653 avg:  0.12438656112288637
point:  40.6

point:  40.6574300004 -73.9471600002 mid:  0.06385725439709633 avg:  0.042571502931397553
point:  40.7630796436 -73.9626851397 mid:  0.09056409487221465 avg:  0.09567146269348369
point:  40.8296624199 -73.9484220401 mid:  0.049509918071345256 avg:  0.04694874813881294
point:  40.8277990001 -73.9285650002 mid:  0.07142988246548776 avg:  0.0755144020989501
point:  40.8537639997 -73.9078989995 mid:  0.08583340612738313 avg:  0.09983654566723531
point:  40.7705449998 -73.9602539997 mid:  0.0953534054747946 avg:  0.10907305448992931
point:  40.6684865101 -73.9336508496 mid:  0.5603697594553609 avg:  0.41598435719342275
point:  40.7243290001 -73.9977020004 mid:  0.22422146480629288 avg:  0.21883827977177214
point:  40.7622099 -73.9818807495 mid:  0.07741026040838786 avg:  0.08260632858607285
point:  40.7325187897 -73.9846943705 mid:  0.05729605554021922 avg:  0.059010696153473395
point:  40.7228539996 -74.0062770002 mid:  0.1893481446084811 avg:  0.15203814643072283
point:  40.6509619998 -73

...

point:  40.8269279998 -73.9498679995 mid:  0.041845917181034566 avg:  0.03811822862050548
point:  40.73369379 -73.9870435095 mid:  0.0453064494470849 avg:  0.04149467107691876
point:  40.7315385703 -73.9832456504 mid:  0.05856160954442706 avg:  0.06322708520193433
point:  40.8397099997 -73.8877570001 mid:  0.13323402136813328 avg:  0.09970510991729638
point:  40.7870788398 -73.9718814398 mid:  0.08498760122408097 avg:  0.07950685593525454
point:  40.7526648201 -73.9728428702 mid:  0.05391081212698025 avg:  0.05788118175966934
point:  40.7102499999 -73.9979799999 mid:  0.0 avg:  0.0
point:  40.8118409998 -73.9427249998 mid:  0.08965026950199127 avg:  0.10486748262230515
point:  40.8051138099 -73.9667880603 mid:  0.09329619961577697 avg:  0.0799424409806424
point:  40.687191 -73.7695589996 mid:  0.0 avg:  0.0
point:  40.5777710002 -74.1023100002 mid:  0.13499272640346793 avg:  0.1229981331080054
point:  40.7252909902 -73.9868423694 mid:  0.09318854474435956 avg:  0.09043124199998781
poin

point:  40.7409980115 -74.0054512019 mid:  0.10493510511708004 avg:  0.12725259936720978
point:  40.760283 -73.9873129997 mid:  0.0827992943243342 avg:  0.06915316562855975
point:  40.8406719997 -73.8426019995 mid:  0.007526079255618591 avg:  0.1548680906729645
point:  40.7942156129 -73.9434510796 mid:  0.08370303556353578 avg:  0.09197309291544169
point:  40.6853080003 -73.9744719998 mid:  0.10692656692346038 avg:  0.0818850048600983
point:  40.6885999996 -74.0145499996 mid:  0.0833591840639216 avg:  0.08969985685253519
point:  40.8327310701 -73.9422657602 mid:  0.05028628734013039 avg:  0.06375707594600025
point:  40.7624639997 -73.9262620002 mid:  0.14876159932510866 avg:  0.13224863186861677
point:  40.8279019997 -73.9283560001 mid:  0.09730054532009874 avg:  0.07562593871972859
point:  40.7534138897 -73.9666514402 mid:  0.09653378722673253 avg:  0.11169394457926991
point:  40.7130013004 -74.0118027004 mid:  0.22958681151147547 avg:  0.1779667334212632
point:  40.7437055002 -73.992

point:  40.7625260003 -73.9679670005 mid:  0.11736449068308237 avg:  0.10859794824840958
point:  40.785868 -73.9689159997 mid:  0.3528912866397896 avg:  0.35109885810752806
point:  40.7451260004 -73.9846389996 mid:  0.07434643929903466 avg:  0.07899430132752604
point:  40.8059740003 -73.9412440005 mid:  0.11389515463575292 avg:  0.10682177250354664
point:  40.6912240004 -73.9866200005 mid:  0.02623551466094974 avg:  0.0434021749663471
point:  40.7225326303 -73.9546886905 mid:  0.2467293799131832 avg:  0.28758229126505486
point:  40.5913570544 -73.92396467 mid:  1.5925641114033628 avg:  1.5953112313467048
point:  40.7046865504 -73.7966203899 mid:  0.09511161924745563 avg:  0.08409578972393598
point:  40.5977150787 -74.0838283464 mid:  0.1541350565550555 avg:  0.20377781262673453
point:  40.7530030498 -73.9792460102 mid:  0.06339325506648856 avg:  0.06080883232078466
point:  40.7972480004 -73.9696809999 mid:  0.04012514008078823 avg:  0.04088478422520419
point:  40.7289209029 -73.9877837

point:  40.7421802504 -73.9187325201 mid:  0.08413578119538279 avg:  0.07188083882700952
point:  40.7663861201 -73.92045929 mid:  0.10016672372802538 avg:  0.10511474978611272
point:  40.8166889202 -73.9465480905 mid:  0.06849814474802908 avg:  0.06141633760952103
point:  40.8135947398 -73.9304002806 mid:  0.08210292475267324 avg:  0.10883437353889659
point:  40.5714370196 -74.1089594206 mid:  0.13897631232640004 avg:  0.11349095026232436
point:  40.76475912 -73.9640994396 mid:  0.13720087788882104 avg:  0.10195183643322221
point:  40.7730029997 -73.9584740003 mid:  0.045035214543371685 avg:  0.04903079613669717
point:  40.77281462 -73.9582204299 mid:  0.035125077969700325 avg:  0.04142325384772765
point:  40.7787919999 -73.9536629999 mid:  0.04216522816964689 avg:  0.036121206553851655
point:  40.82755199 -73.9259025195 mid:  0.07869258698784738 avg:  0.06963980028497131
point:  40.8145045398 -73.9593700004 mid:  0.36042869353545237 avg:  0.33167946804207427
point:  40.8611500003 -73.

point:  40.7173040001 -73.9568719995 mid:  0.14029831837138834 avg:  0.18422788280609326
point:  40.718092 -73.9998919998 mid:  0.07096506414168414 avg:  0.059258176523984095
point:  40.722301 -73.9971410001 mid:  0.25605526839493864 avg:  0.279693406309553
point:  40.6806299998 -73.9953819995 mid:  0.05197505131303995 avg:  0.07346988060963486
point:  40.8108964904 -73.9214341701 mid:  0.16828639211207153 avg:  0.14934258128677483
point:  40.7019303441 -73.9422392381 mid:  0.04028380774824176 avg:  0.03991610592544742
point:  40.6385596088 -73.9536032378 mid:  0.5981841360301386 avg:  0.5329469765266206
point:  40.6868115694 -73.9849696135 mid:  0.24901920645150497 avg:  0.22828873451967302


#### Question 1.8: If you plot the number of hotspot activations for each month, you'll notice a general increase but then a precipitous drop after June 2018. Using a linear estimate for the number of monthly activations, what is rate of increase in monthly activations? Only consider data before July 1, 2018 and set the start date as the earliest date of the data. If you need to, use 30.5 days in a month.  (10 pts)


In [11]:
#export
# Code script for Q 1.8
# Write your code script below 

month_dict = {}
dead_line = time.strptime("2018-07", "%Y-%m")
dates = df["Activated"]
for date in dates:
    month, day, year = date.split("/")
    curr_date_str = year + "-" + month
    curr_date = time.strptime(curr_date_str, "%Y-%m")
    if curr_date < dead_line:
        if curr_date_str in month_dict:
            month_dict[curr_date_str] += 1
        else:
            month_dict[curr_date_str] = 1
    
sorted_month_dict = sorted(month_dict.items(), key=lambda d: d[0])

size = len(sorted_month_dict)
for i in range(0, size-1):
    last_month, last_times = sorted_month_dict[i]
    curr_month, curr_times = sorted_month_dict[i+1]
    # a[i+1] - a[i] / a[i]
    rate = (int(curr_times)-int(last_times)) / int(last_times)
    print("current month", curr_month, " current times", curr_times)
    print("last month", last_month, " last times", last_times)
    print("rate: ", rate*100, "%")
    print("================")

current month 2016-02  current times 15
last month 2016-01  last times 11
rate:  36.36363636363637 %
current month 2016-03  current times 64
last month 2016-02  last times 15
rate:  326.6666666666667 %
current month 2016-04  current times 57
last month 2016-03  last times 64
rate:  -10.9375 %
current month 2016-05  current times 27
last month 2016-04  last times 57
rate:  -52.63157894736842 %
current month 2016-06  current times 26
last month 2016-05  last times 27
rate:  -3.7037037037037033 %
current month 2016-07  current times 103
last month 2016-06  last times 26
rate:  296.1538461538462 %
current month 2016-08  current times 20
last month 2016-07  last times 103
rate:  -80.58252427184466 %
current month 2016-09  current times 45
last month 2016-08  last times 20
rate:  125.0 %
current month 2016-10  current times 46
last month 2016-09  last times 45
rate:  2.2222222222222223 %
current month 2016-11  current times 45
last month 2016-10  last times 46
rate:  -2.1739130434782608 %
cu

### Now, please fill out the all numeric answers to Question 1.1-Question 1.8 in the following code cells. DO NOT UNCOMMENT.

In [ ]:
#export
# Your answer to ***Question 1.1***:
17

In [ ]:
#export
# Your answer to ***Question 1.2***:
3.5854172943657723 %

In [ ]:
#export
# Your answer to ***Question 1.3***:
60.43333333333333

In [ ]:
#export
# Your answer to ***Question 1.4***:
0.24368825466520308

In [ ]:
#export
# Your answer to ***Question 1.5***:
120

In [ ]:
#export
# Your answer to ***Question 1.6***:
0.8122383252818035

In [ ]:
#export
# Your answer to ***Question 1.7***:

In [ ]:
#export
# Your answer to ***Question 1.8***:
326.6666666666667 %

# Section 2: Simple Programming

Consider $n$ buildings on a grid at positions $0$ to $n+1$. Each building has a height $h_{i}$. You can arrange the buildings in any order but you must leave the slot at position 0 open. Imagine a laser is shot just below the roof and to left of a building. The laser travels any number of grid points until it encounters a building of the same height or taller or reaches the end of the grid, position $0$. 

For example, consider 4 buildings of height $3, 3, 4,$ and $1$ arranged at grid points $1, 2, 3$, and $4$, respectively. The laser travels $1, 1, 3$, and $1$ grid points for each of the buildings, respectively. Let's call the sum of the lasers' distances $V$. For this example, $V=6$. For all questions, give your answer to $10$ places after the decimal point.

### Please answer the following 5 questions. For each question,
1. Please write down the script used to compute your response in the ***Code*** Cell; Conclude your scripts with a final ***print()*** function to print out final numeric answers. **Important: Make sure your scripts are executable !!!**
2. Please fill out the final numeric answers in the cells at the end of the section.

#### Question 2.1: Consider 4 buildings of heights 1, 1, 3, and 4. For all possible configurations, what is the mean of sum of distances V? (10 pts)

In [12]:
#export
# Code script for Q 2.1
# Write your code script below 

array = [1,1,3,4]
pailie = set(list(itertools.permutations(array)))
t = 0
res = []
for dist in pailie:
    dist = list(dist)
    dist.insert(0, 0) # = [0,1,1,3,4]
    size = len(dist)
    V = 0
    for r in range(size-1, 0, -1):
        for l in range(r-1, 0, -1):
            if dist[r] <= dist[l]:
                break
        V += (r-l)
    res.append(V)

print(np.average(res))

4.166666666666667


#### Question 2.2: Consider 10 buildings of heights 1, 2, 3, ..., 10. For all possible configurations, what is the mean of sum of distances V?  (10 pts)

In [5]:
#export
# Code script for Q 2.2
# Write your code script below

array = []

for i in range(1, 11):
    array.append(i)

pailie = set(list(itertools.permutations(array)))
t = 0
res = []
for dist in pailie:
    dist = list(dist)
    dist.insert(0, 0) # = [0,1,1,3,4]
    size = len(dist)
    V = 0
    for r in range(size-1, 0, -1):
        for l in range(r-1, 0, -1):
            if dist[r] <= dist[l]:
                break
        V += (r-l)
    res.append(V)

print(np.average(res))

19.28968253968254


#### Question 2.3: Consider 20 buildings of heights 1, 2, 3, ..., 20. For all possible configurations, what is the mean of sum of distances $V$? The calculation of the standard deviation is not needed.  (10 pts)

In [ ]:
#export
# Code script for Q 2.3
# Write your code script below 
array = []

for i in range(1, 21):
    array.append(i)

pailie = set(list(itertools.permutations(array)))
t = 0
res = []
for dist in pailie:
    dist = list(dist)
    dist.insert(0, 0) # = [0,1,1,3,4]
    size = len(dist)
    V = 0
    for r in range(size-1, 0, -1):
        for l in range(r-1, 0, -1):
            if dist[r] <= dist[l]:
                break
        V += (r-l)
    res.append(V)

print(np.average(res))

#### Question 2.4: Consider 4 buildings of heights 1, 1, 3, and 4. For all possible configurations, what is the standard deviation of the sum of distances $V$?  (10 pts)

In [7]:
#export
# Code script for Q 2.4
# Write your code script below  

array = [1,1,3,4]
pailie = set(list(itertools.permutations(array)))
t = 0
res = []
for dist in pailie:
    dist = list(dist)
    dist.insert(0, 0) # = [0,1,1,3,4]
    size = len(dist)
    V = 0
    for r in range(size-1, 0, -1):
        for l in range(r-1, 0, -1):
            if dist[r] <= dist[l]:
                break
        V += (r-l)
    res.append(V)

print(np.std(res))

0.8975274678557508


#### Question 2.5: Consider 10 buildings of heights 1, 2, 3, ..., 10. For all possible configurations, what is the standard deviation of the sum of distances $V$?  (10 pts)

In [8]:
#export
# Code script for Q 2.5
# Write your code script below  
array = []

for i in range(1, 11):
    array.append(i)

pailie = set(list(itertools.permutations(array)))
t = 0
res = []
for dist in pailie:
    dist = list(dist)
    dist.insert(0, 0) # = [0,1,1,3,4]
    size = len(dist)
    V = 0
    for r in range(size-1, 0, -1):
        for l in range(r-1, 0, -1):
            if dist[r] <= dist[l]:
                break
        V += (r-l)
    res.append(V)


print(np.std(res))

3.9665532069623617


### Now, please fill out the all numeric answers to Question 2.1-Question 2.5 in the following code cells. DO NOT UNCOMMENT.

In [ ]:
#export
# Your answer to ***Question 2.1***:
4.1666666667

In [ ]:
#export
# Your answer to ***Question 2.2***:
19.2896825397

In [ ]:
#export
# Your answer to ***Question 2.3***:

In [ ]:
#export
# Your answer to ***Question 2.4***:
0.8975274679

In [ ]:
#export
# Your answer to ***Question 2.5***:
3.9665532069

## Congratulations for finishing the first part of the final exam !
### Remeber to save your Jupyter Notebook.
### Now it's a good time to submit for grading.

Please uncomment and run the cell below. Your code will be generated in the folder named `first_part`, please upload `submission.py` file AND the Jupyter Notebook (`AcF_351b_Python_Stream_Final_Exam_Part_I.ipynb`) together.

In [16]:
%run notebook2script1 first_part

Converted AcF_351b_Python_Stream_Final_Exam_Part_I.ipynb to first_part\submission_part_I.py
